## Introduction
This notebook runs all the processing steps one by one for several models and renders the output. Each section is individually runnable after a kernel restart 

## Observations
* Symbolic tracing did not play well with any BERT model, because it creates proxies for mutually exclusive inputs to e.g. `DistilBertModel.forward`
  * This was fixed by making the `concrete_args` input to `fx.symbolic_trace` available to the `MAV` and `MavTracer` objects
  * For BERT models, `concrete_args={'inputs_embeds':None}` gets around this issue
* Still, most NLP models use proxy variables for control flow, which is not supported by `torch.fx`
  * Perhaps fixing more arguments via `concrete_args` could work around this. To be investigated.

## Wav2Vec

In [2]:
import sys
sys.path.append('..')
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import torch
from idlmav import MAV, plotly_renderer

model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model.eval()
inputs = torch.randn(1, 16000)
device = 'cpu'

mav = MAV(model, inputs)
with plotly_renderer('notebook_connected'): mav.show_figure()

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:312: UserWarning:

Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.



Tracing failed with torch.fx.symbolic_trace: zeros() received an invalid combination of arguments - got (tuple, device=Attribute, dtype=Attribute), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)

Tracing with torch.compile


## Whisper tiny encoder

In [1]:
import sys
sys.path.append('..')
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from idlmav import MAV, plotly_renderer

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model.eval()
inputs = torch.randn((1,80,3000))
device = 'cpu'

mav = MAV(model.model.encoder, inputs)
with plotly_renderer('notebook_connected'): mav.show_figure()

Tracing failed with torch.fx.symbolic_trace: symbolically traced variables cannot be used as inputs to control flow
Tracing with torch.compile


INFO:2025-02-13 23:03:16 5252:5252 init.cpp:181] If you see CUPTI_ERROR_INSUFFICIENT_PRIVILEGES, refer to https://developer.nvidia.com/nvidia-development-tools-solutions-err-nvgpuctrperm-cupti


## Audio Spectrogram Transformer (AST)

In [3]:
import sys
sys.path.append('..')
from transformers import ASTConfig, ASTModel
import torch
from idlmav import MAV, plotly_renderer

configuration = ASTConfig()
model = ASTModel(configuration)
model.eval()
inputs = torch.randn(1, 1, 128, 128)
device = 'cpu'

mav = MAV(model, inputs)
with plotly_renderer('notebook_connected'): mav.show_figure()

Tracing failed with torch.fx.symbolic_trace: symbolically traced variables cannot be used as inputs to control flow
Tracing with torch.compile


TorchRuntimeError: Failed running call_function <built-in method conv2d of type object at 0x7f8acaa5f1c0>(*(FakeTensor(..., size=(1, 1, 128, 1, 128)), Parameter(FakeTensor(..., size=(768, 1, 16, 16), requires_grad=True)), Parameter(FakeTensor(..., size=(768,), requires_grad=True)), (10, 10), (0, 0), (1, 1), 1), **{}):
Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [1, 1, 128, 1, 128]

from user code:
   File "/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/transformers/models/audio_spectrogram_transformer/modeling_audio_spectrogram_transformer.py", line 546, in forward
    embedding_output = self.embeddings(input_values)
  File "/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/transformers/models/audio_spectrogram_transformer/modeling_audio_spectrogram_transformer.py", line 76, in forward
    embeddings = self.patch_embeddings(input_values)
  File "/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/transformers/models/audio_spectrogram_